## *Student Names with IDs*


---

*   Bushra Talaq Al-Sulami - **Leader** : 444001895
*   Heba Faisal Al-Sharif : 443002516

*   Mariam Hassan Battah : 443006035
*   Layali Saad Al-Saeedi : 444002078


---


# **Phase 1: Relational Database**
- Design a normalized relational schema
- Create tables and insert sample data
- Perform CRUD operations using SQL and Python
- Apply indexing and query optimization

In [1]:
import pandas as pd
import sqlite3 # to run SQL inside Python

In [2]:
from google.colab import files
uploaded = files.upload() # uploade the Dataset into our Colab

Saving ProjectDataSet.csv to ProjectDataSet.csv


In [3]:
df = pd.read_csv('ProjectDataSet.csv')
df.head() # Show first 5 rows

,customer_id,product_id,basket_date,basket_count
0,42366585,41475073,6/19/2019,2
1,35956841,43279538,6/19/2019,2
2,26139578,31715598,6/19/2019,3
3,3262253,47880260,6/19/2019,2
4,20056678,44747002,6/19/2019,2


## Explore and Clean the Data

In [4]:
df.info() # To shows the column names, data types, number of non-null entries
df.describe(include='all') # To show statistics about dataset
df.isnull().sum() # To show how many missing values are in each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   15000 non-null  int64 
 1   product_id    15000 non-null  int64 
 2   basket_date   15000 non-null  object
 3   basket_count  15000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 468.9+ KB


,0
customer_id,0
product_id,0
basket_date,0
basket_count,0


In [5]:
df.dropna(inplace=True) # delete empty cells
df.drop_duplicates(inplace=True) # delete duplicates

## Normalize the Data
into 3 Tables

In [16]:
# Extract unique customers and products
customers = df[['customer_id']].drop_duplicates() # list of unique customers
products = df[['product_id']].drop_duplicates()

# Prepare basket transactions, it stores: customer bought what, which day and how many
baskets = df[['customer_id', 'product_id', 'basket_date', 'basket_count']].copy()
baskets.reset_index(inplace=True)
baskets.rename(columns={'index': 'basket_id'}, inplace=True) # add basket_id to be the primary key

## Create SQLite DB and Tables + Insert Data into Tables

In [7]:
conn = sqlite3.connect('ecommerce.db') #creates a SQLite database file called ecommerce.db
cursor = conn.cursor() # The SQL command executor

In [8]:

cursor.execute('''
CREATE TABLE customers (
    customer_id TEXT PRIMARY KEY
)
''')

cursor.execute('''
CREATE TABLE products (
    product_id TEXT PRIMARY KEY
)
''')

cursor.execute('''
CREATE TABLE baskets (
    basket_id INTEGER PRIMARY KEY,
    customer_id TEXT,
    product_id TEXT,
    basket_date TEXT,
    basket_count INTEGER,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')
conn.commit()

In [9]:
# These lines take the Pandas DataFrames and insert the data into the corresponding SQL tables.

customers.to_sql('customers', conn, if_exists='append', index=False)
products.to_sql('products', conn, if_exists='append', index=False)
baskets.to_sql('baskets', conn, if_exists='append', index=False)

# if_exists='append': don’t overwrite the table, just add to it.
#index=False: don’t insert the DataFrame index as a column.

15000

## CRUD Operations

In [10]:
# Create
cursor.execute("""
INSERT INTO customers (customer_id) VALUES ('C999')
""")
conn.commit() # saves the change

In [11]:
# Read
pd.read_sql_query("SELECT * FROM baskets LIMIT 5", conn) # Shows the first 5 rows from baskets

,basket_id,customer_id,product_id,basket_date,basket_count
0,0,42366585,41475073,6/19/2019,2
1,1,35956841,43279538,6/19/2019,2
2,2,26139578,31715598,6/19/2019,3
3,3,3262253,47880260,6/19/2019,2
4,4,20056678,44747002,6/19/2019,2


In [12]:
# Update
cursor.execute("""
UPDATE baskets SET basket_count = 99 WHERE basket_id = 0
""")
conn.commit()

In [13]:
# Delete
cursor.execute("DELETE FROM customers WHERE customer_id = 'C999'")
conn.commit()

##  Apply indexing and query optimization

In [14]:
#create indexes to speed up searches by customer_id or product_id in the baskets table
cursor.execute("CREATE INDEX idx_customer_id ON baskets(customer_id)")
cursor.execute("CREATE INDEX idx_product_id ON baskets(product_id)")
conn.commit()

In [15]:
pd.read_sql_query("SELECT * FROM baskets WHERE customer_id = 'C1000'", conn) #Searches for transactions by customer 'C1000'

,basket_id,customer_id,product_id,basket_date,basket_count


# **Phase 2:**
- NoSQL Database
- Transform part of the data into a document or key-value format
- Insert data using a Python-based NoSQL library (e.g., pymongo)
- Perform basic queries

**Step 1: Install TinyDB**

In [17]:
!pip install tinydb


**Step 2: Import and Create TinyDB Database**

In [18]:
from tinydb import TinyDB, Query

# Create or connect to TinyDB database
db = TinyDB('baskets_nosql.json')
Basket = Query()
print('Connected to TinyDB!')

Connected to TinyDB!


**Step 3: Convert a Few Basket Records into**

In [19]:
# Insert 3 sample basket records
if len(db) == 0:
    sample_baskets = [
        {'basket_id': 0, 'customer_id': 42366585, 'product_id': 41475073, 'basket_date': '6/19/2019', 'basket_count': 2},
        {'basket_id': 1, 'customer_id': 35956841, 'product_id': 43279538, 'basket_date': '6/19/2019', 'basket_count': 2},
        {'basket_id': 2, 'customer_id': 26139578, 'product_id': 31715598, 'basket_date': '6/19/2019', 'basket_count': 3}
    ]
    db.insert_multiple(sample_baskets)
print('Inserted sample baskets!')

Inserted sample baskets!


**Step 4: Query All Documents**

In [20]:
print('All basket documents:')
for item in db.all():
    print(item)

All basket documents:
{'basket_id': 0, 'customer_id': 42366585, 'product_id': 41475073, 'basket_date': '6/19/2019', 'basket_count': 2}
{'basket_id': 1, 'customer_id': 35956841, 'product_id': 43279538, 'basket_date': '6/19/2019', 'basket_count': 2}
{'basket_id': 2, 'customer_id': 26139578, 'product_id': 31715598, 'basket_date': '6/19/2019', 'basket_count': 3}


**Step 5: Query Specific Data**

In [21]:
print("Baskets by customer 35956841:")
for item in db.search(Basket.customer_id == 35956841):
    print(item)

Baskets by customer 35956841:
{'basket_id': 1, 'customer_id': 35956841, 'product_id': 43279538, 'basket_date': '6/19/2019', 'basket_count': 2}


**Step 6: Update a Document**

In [22]:
db.update({'basket_count': 99}, Basket.basket_id == 0)
print("Updated basket_count to 99 for basket_id = 0")

Updated basket_count to 99 for basket_id = 0


**Step 7: Delete a Document**

In [23]:
db.remove(Basket.basket_id == 0)
print("Deleted basket with basket_id = 0")

Deleted basket with basket_id = 0


**Step 8: View remaining**

In [24]:
print("Remaining documents in NoSQL DB:")
for item in db.all():
    print(item)

Remaining documents in NoSQL DB:
{'basket_id': 1, 'customer_id': 35956841, 'product_id': 43279538, 'basket_date': '6/19/2019', 'basket_count': 2}
{'basket_id': 2, 'customer_id': 26139578, 'product_id': 31715598, 'basket_date': '6/19/2019', 'basket_count': 3}


# **Phase 3:**
- Stream Processing
- Simulate a data stream (e.g., JSON or CSV)
- Use PySpark to filter and process the data
- Save or display the processed output

Step 1: Install PySpark

In [25]:
!pip install -q pyspark

Step 2: Start SparkSession

In [26]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("StudentStreamLab").getOrCreate()

Step 3: Load dataset

In [27]:
# Read the dataset from a CSV file
df = spark.read.option("header", True).csv("ProjectDataSet.csv", inferSchema=True)
#show first rows
df.show(5)

+-----------+----------+-----------+------------+
|customer_id|product_id|basket_date|basket_count|
+-----------+----------+-----------+------------+
|   42366585|  41475073|  6/19/2019|           2|
|   35956841|  43279538|  6/19/2019|           2|
|   26139578|  31715598|  6/19/2019|           3|
|    3262253|  47880260|  6/19/2019|           2|
|   20056678|  44747002|  6/19/2019|           2|
+-----------+----------+-----------+------------+
only showing top 5 rows



 Step 4: Filter data (basket_count)

In [28]:
#Filter customers who bought more than 2 items in a basket
filtered_df = df.filter(df["basket_count"] > 2)
filtered_df.show()
purchase_count = filtered_df.count()
print(f"Number of purchases with more than 2 items: {purchase_count}")

+-----------+----------+-----------+------------+
|customer_id|product_id|basket_date|basket_count|
+-----------+----------+-----------+------------+
|   26139578|  31715598|  6/19/2019|           3|
|    5216868|   8670770|  6/19/2019|           6|
|   28204561|  39058017|  6/18/2019|           3|
|    7751651|  46500920|  6/16/2019|           3|
|   25083717|  31546667|  6/16/2019|           3|
|   27641594|  33506465|  6/16/2019|           3|
|    4749863|  33354168|  6/16/2019|           3|
|    1227904|  43482185|  6/16/2019|           3|
|   12376998|  50836009|  6/16/2019|           3|
|   29816325|  42244011|  6/15/2019|           3|
|   14953708|  49860049|  6/15/2019|           3|
|   26633229|  33028936|  6/15/2019|           3|
|   27507650|   4915558|  6/15/2019|           3|
|   35960037|  31541857|  6/15/2019|           3|
|    2469408|  42383131|  6/15/2019|           3|
|   11004573|  50596116|  6/15/2019|           3|
|   39042643|  39359897|  6/14/2019|           3|


Step 5: Save Filtered Data to CSV

In [29]:
# Save filtered data to local Colab folder
output_path = "/content/output_customers"
filtered_df.write.mode("overwrite").csv(output_path)

# Verify file was saved
import os
print("Files saved to:", os.listdir(output_path))

Files saved to: ['part-00000-3d567123-2286-4b1b-aba7-15e7a2c0db90-c000.csv', '.part-00000-3d567123-2286-4b1b-aba7-15e7a2c0db90-c000.csv.crc', '_SUCCESS', '._SUCCESS.crc']
